In [6]:
pip install image

Processing /Users/dang/Library/Caches/pip/wheels/56/88/e6/897194cfe8c08a8b9afd881d3bf53d102e13fa39607d721383/image-1.5.33-py2.py3-none-any.whl
  Using cached Django-3.2.3-py3-none-any.whl (7.9 MB)
  Using cached Pillow-8.2.0-cp37-cp37m-macosx_10_10_x86_64.whl (2.8 MB)
  Using cached sqlparse-0.4.1-py3-none-any.whl (42 kB)
  Using cached asgiref-3.3.4-py3-none-any.whl (22 kB)
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
from PIL import Image, ImageDraw
import pandas as pd

In [8]:
hex_data  = pd.read_csv(r"./colordata/Hex_data.csv")

In [9]:
hex_data.head()

,Material,name,Hex,Hex_primary
0,Solid,White,F4F4F4,#F4F4F4
1,Solid,Grey,8A928D,#8A928D
2,Solid,Light Yellow,FFD67F,#FFD67F
3,Solid,Brick Red,F2705E,#F2705E
4,Solid,Brick Yellow,B0A06F,#B0A06F


In [11]:
def convert_hex_to_rgb(hex):
    return tuple(int(hex[i:i+2], 16) for i in (0, 2, 4))
hex_data['rgb'] = hex_data['Hex'].map(convert_hex_to_rgb)

In [12]:
some_color_2 = list(hex_data['rgb'])

In [13]:
hex_data.to_csv("./colordata/hex_data_new.csv")

In [14]:
hex_data.head()

,Material,name,Hex,Hex_primary,rgb
0,Solid,White,F4F4F4,#F4F4F4,"(244, 244, 244)"
1,Solid,Grey,8A928D,#8A928D,"(138, 146, 141)"
2,Solid,Light Yellow,FFD67F,#FFD67F,"(255, 214, 127)"
3,Solid,Brick Red,F2705E,#F2705E,"(242, 112, 94)"
4,Solid,Brick Yellow,B0A06F,#B0A06F,"(176, 160, 111)"


In [15]:
def image_to_dots(image_path: str,
                  pixelization_length=10,
                  mapping_color=False
                 ):
    
    img = Image.open(image_path)
    pixels = img.load()

    # Choose the pixelization length
    pixelization_length = 10

    # Define the pixelize and mask image
    pixelize_image = Image.new('RGBA', img.size)
    mask_image = Image.new('RGBA', img.size, 'black')

    x_units = int(img.size[0] / pixelization_length)
    y_units = int(img.size[1] / pixelization_length)

    draw = ImageDraw.Draw(pixelize_image)
    mask_draw = ImageDraw.Draw(mask_image)

    for i in range(0, x_units):
        for j in range(0, y_units):
            mask_draw.ellipse((i * pixelization_length, j * pixelization_length,
                      i * pixelization_length + pixelization_length - 1,
                      j * pixelization_length + pixelization_length - 1),
                     (255,255,255,0))

            total_red_intensity = total_green_intensity = total_blue_intensity = 0
            averaging_pixel_number = pixelization_length * pixelization_length

            for k in range(0, pixelization_length):
                for l in range(0, pixelization_length):
                    total_red_intensity += pixels[i * pixelization_length + k, j * pixelization_length + l][0]
                    total_green_intensity += pixels[i * pixelization_length + k, j * pixelization_length + l][1]
                    total_blue_intensity += pixels[i * pixelization_length + k, j * pixelization_length + l][2]

            average_red_intensity = int(total_red_intensity / averaging_pixel_number)
            average_green_intensity = int(total_green_intensity / averaging_pixel_number)
            average_blue_intensity = int(total_blue_intensity / averaging_pixel_number)

            if mapping_color is True:
                some_color_1 = [(average_red_intensity, average_green_intensity, average_blue_intensity)]
                closest_idx = scipy.spatial.distance.cdist(some_color_1,some_color_2).argmin(1)
                data_in_palette = some_color_2[closest_idx[0]]
                average_red_intensity_new, average_green_intensity_new, average_blue_intensity_new = tuple(data_in_palette)
                draw.rectangle((i * pixelization_length, j * pixelization_length,
                                i * pixelization_length + pixelization_length - 1,
                                j * pixelization_length + pixelization_length - 1),
                               (average_red_intensity_new, average_green_intensity_new, average_blue_intensity_new)
                              )
            else:
                draw.rectangle((i * pixelization_length, j * pixelization_length,
                                i * pixelization_length + pixelization_length - 1,
                                j * pixelization_length + pixelization_length - 1),
                               (average_red_intensity, average_green_intensity, average_blue_intensity)
                              )

    # Save the intermediate images for demo purpose
    mask_image.save("mask.png")
    pixelize_image = pixelize_image.convert('RGB')
    pixelize_image.save("pixelize.jpg")

    pixelize_image.paste(mask_image, mask=mask_image)
    pixelize_image = pixelize_image.convert('RGB')
    pixelize_image.save("result.png")
    pixelize_image.show()

In [17]:
img = "./testimage/1.jpg"
image_to_dots(image_path=img, pixelization_length=10, mapping_color=False)